--------

In [1]:
import os
import cv2
import numpy as np
import pickle
from deepface import DeepFace
from pathlib import Path
from datetime import datetime
from collections import Counter

In [2]:
# Global Config
DATASET_DIR = "faces"          # Folder containing one image per student
EMBEDDINGS_FILE = "embeddings_db.pkl"
MODEL_NAME = "ArcFace"           # High accuracy pretrained model
INPUT_SIZE = (160, 160)
RECOG_THRESHOLD = 0.45

#### Preprocessing

In [4]:
def augment_image(img):
    """Generate a few augmentations for one face image."""
    aug_list = [img]
    
    # Horizontal flip
    aug_list.append(cv2.flip(img, 1))
    
    # Brightness variations
    for alpha in [0.8, 1.2]:
        bright = cv2.convertScaleAbs(img, alpha=alpha, beta=10)
        aug_list.append(bright)
    
    # Small rotations
    h, w = img.shape[:2]
    for angle in [-10, 10]:
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
        rotated = cv2.warpAffine(img, M, (w, h))
        aug_list.append(rotated)
    
    return aug_list

def histogram_equalize_color(bgr_img):
    """Equalize the Y channel of the YCrCb color space."""
    ycrcb = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycrcb)
    y_eq = cv2.equalizeHist(y)
    merged = cv2.merge([y_eq, cr, cb])
    return cv2.cvtColor(merged, cv2.COLOR_YCrCb2BGR)

def align_and_resize(face_img, required_size=INPUT_SIZE):
    img_eq = histogram_equalize_color(face_img)
    return cv2.resize(img_eq, required_size)

In [5]:
# Utility Functions for Similarity and Recognition

def cosine_similarity(a, b):
    a, b = np.array(a), np.array(b)
    num = np.dot(a, b)
    den = np.linalg.norm(a) * np.linalg.norm(b)
    return num / den if den != 0 else 0.0

#### Build Embeddings Database (Single Image per Student)

In [7]:
def build_embeddings_db(dataset_path=DATASET_DIR, model_name=MODEL_NAME, out_file=EMBEDDINGS_FILE):
    embeddings_db = {}
    people = [p for p in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, p))]
    print("Building embeddings for:", people)
    
    for person in people:
        folder = os.path.join(dataset_path, person)
        img_files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
        if len(img_files) == 0:
            continue
        
        img_path = os.path.join(folder, img_files[0])
        img = cv2.imread(img_path)
        aug_imgs = augment_image(img)
        vectors = []
        
        for aug in aug_imgs:
            rep = DeepFace.represent(img_path=aug, model_name=model_name, enforce_detection=False)
            if isinstance(rep, list) and len(rep) > 0:
                vectors.append(rep[0]['embedding'])
        
        avg_emb = np.mean(vectors, axis=0)
        embeddings_db[person] = avg_emb
        print(f"Created augmented embedding for {person} ({len(aug_imgs)} variants)")
    
    with open(out_file, "wb") as f:
        pickle.dump(embeddings_db, f)
    print("✅ Saved embeddings database:", out_file)
    return embeddings_db

embeddings = build_embeddings_db()

Building embeddings for: ['.ipynb_checkpoints', 'Surya']
Created augmented embedding for Surya (6 variants)
✅ Saved embeddings database: embeddings_db.pkl


In [8]:
def load_embeddings_db(path=EMBEDDINGS_FILE):
    with open(path, "rb") as f:
        db = pickle.load(f)
    print("Loaded embeddings for:", list(db.keys()))
    return db

#### Recognition

In [10]:
def get_face_embedding(face_img, model_name=MODEL_NAME):
    try:
        rep = DeepFace.represent(img_path=face_img, model_name=model_name, enforce_detection=False)
        if isinstance(rep, list) and len(rep) > 0 and 'embedding' in rep[0]:
            return np.array(rep[0]['embedding'], dtype=np.float32)
    except Exception as e:
        print("Embedding error:", e)
    return None

def recognize_face(face_img, embeddings_db):
    emb = get_face_embedding(face_img)
    if emb is None:
        return "Unknown", 0.0
    best_name, best_score = "Unknown", -1.0
    for name, db_emb in embeddings_db.items():
        sim = cosine_similarity(emb, db_emb)
        if sim > best_score:
            best_score, best_name = sim, name
    if best_score < RECOG_THRESHOLD:
        best_name = "Unknown"
    return best_name, best_score

#### Real-Time Recognition + Tracking

In [12]:
def bbox_iou(boxA, boxB):
    xA, yA = max(boxA[0], boxB[0]), max(boxA[1], boxB[1])
    xB, yB = min(boxA[2], boxB[2]), min(boxA[3], boxB[3])
    interW, interH = max(0, xB-xA), max(0, yB-yA)
    interArea = interW * interH
    boxAArea = (boxA[2]-boxA[0]) * (boxA[3]-boxA[1])
    boxBArea = (boxB[2]-boxB[0]) * (boxB[3]-boxB[1])
    return interArea / (boxAArea + boxBArea - interArea + 1e-6)

def run_realtime_recognition(embeddings_db, source=0, detection_interval=10):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        raise RuntimeError("Cannot open source:", source)
    
    trackers = {}
    recognition_buffer = {}
    next_id = 0
    frame_count = 0
    
    print("Starting stream... Press 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        orig = frame.copy()
        
        # Update existing trackers
        for tid, (tracker, label, last_seen) in list(trackers.items()):
            ok, bbox = tracker.update(frame)
            if not ok:
                del trackers[tid]
                continue
            x, y, w, h = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
            trackers[tid] = (tracker, label, frame_count)
        
        # Re-detect every few frames
        if frame_count % detection_interval == 0:
            gray = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(60,60))
            for (x, y, w, h) in faces:
                new_box = (x, y, x+w, y+h)
                overlap = any(bbox_iou(new_box, (int(t[0].getROI()[0]), int(t[0].getROI()[1]), 
                                                int(t[0].getROI()[0])+int(t[0].getROI()[2]),
                                                int(t[0].getROI()[1])+int(t[0].getROI()[3]))) > 0.3
                              for t in trackers.values())
                if overlap:
                    continue
                
                face_crop = orig[y:y+h, x:x+w]
                if face_crop.size == 0:
                    continue
                
                face_proc = align_and_resize(face_crop)
                name, score = recognize_face(face_proc, embeddings_db)
                
                # Smooth recognition using buffer
                recognition_buffer.setdefault(next_id, []).append(name)
                if len(recognition_buffer[next_id]) > 5:
                    recognition_buffer[next_id].pop(0)
                label = Counter(recognition_buffer[next_id]).most_common(1)[0][0]
                
                tracker = cv2.TrackerCSRT_create()
                tracker.init(orig, (x, y, w, h))
                trackers[next_id] = (tracker, label, frame_count)
                next_id += 1
        
        cv2.imshow("Attendance - Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [13]:
embeddings = build_embeddings_db()

Building embeddings for: ['.ipynb_checkpoints', 'Surya']
Created augmented embedding for Surya (6 variants)
✅ Saved embeddings database: embeddings_db.pkl


In [23]:
embeddings = load_embeddings_db()
run_realtime_recognition(embeddings, source=0, detection_interval=10)

Loaded embeddings for: ['Surya']
Starting stream... Press 'q' to quit.


AttributeError: 'cv2.TrackerCSRT' object has no attribute 'getROI'

In [16]:
esp_stream_url = "http://192.168.1.50:81/stream"  # example
embeddings = load_embeddings_db()
run_realtime_recognition(embeddings, source=esp_stream_url, detection_interval=8)

Loaded embeddings for: ['Surya']


RuntimeError: ('Cannot open source:', 'http://192.168.1.50:81/stream')